# Quantum gates and simple circuits

Author: Alexandra Semposki

Maintainer: Alexandra Semposki

Now that you have established a working understanding of qubits, you will see below how to use them with quantum gates in simple circuits of one to two qubits. Later on, we can tackle more complicated circuits, but for now we just need a basic knowledge of how to read a quantum circuit and how to follow the qubits through the actions performed on them by the quantum gates. Being able to quickly read a quantum circuit is a wonderful skill to have and will be highly beneficial if you go on to read state of the art quantum computing research articles, where circuits can get very complex *very* rapidly.

## Two-qubit states

Before jump into the operators, gates, and simple circuits to come, we first look at the two-qubit state, a step up from the last lesson where we only dealt with a single qubit. 

Two qubit states will involve four total computational basis states, since we can combine two qubits of $|0\rangle$ and $|1\rangle$ into four different two-qubit states, such as $|01\rangle$. The full expression for the two-qubit state is given as

$$
|\psi\rangle = \alpha |00\rangle + \beta |01\rangle + \gamma |10\rangle + \delta |11\rangle,
$$

where the normalization of the state, if we only measured one qubit of the two, now is given as [1]

$$
\sum_{x\in\{0,1\}^{2}} |c_{x}|^{2} = 1,
$$

where $c$ contains all coefficients in $|\psi\rangle$ ($c \in \{\alpha, \beta, \gamma, \delta\}$), and $x$ here is a set of two numbers where either number is 0 or 1 [1]. This will become clearer when we get to two-qubit Bell states (see the section [Example: Bell states](#bell_states) below). 

## Quantum operators: from matrices to gates

< put Pauli operators >

In [ ]:
# import Qiskit
from qiskit import *

## Quantum gates

< put the Hadamard, CNOT, Toffoli, etc. >

## A few simple circuits

< do the simplest circuits that transform into one another, etc. >

## Example: Bell states

<a id='bell_states'></a>

Let's look at a simple two-qubit circuit where we prepare the four Bell states below. 

$$ 
|\Phi^{+}\rangle = \frac{|00\rangle + |11\rangle}{\sqrt{2}}
$$

$$
|\Phi^{-}\rangle = \frac{|00\rangle - |11\rangle}{\sqrt{2}}
$$

$$
|\Psi^{+}\rangle = \frac{|01\rangle + |10\rangle}{\sqrt{2}}
$$

$$
|\Psi^{-}\rangle = \frac{|01\rangle - |10\rangle}{\sqrt{2}}
$$

What are these __Bell states__? What purpose do they have? Well, they use the Hadamard and CNOT gates we just discussed individually above, and they have some very interesting properties, including quantum teleportation! They play a very important role in the EPR paradox and the recent 2022 Physics Nobel Prize (see [this comprehensive YouTube video](https://www.youtube.com/embed/9OM0jSTeeBg) from IBM Quantum for more information on Bell states and Bell's inequality).

To prepare our circuits, we set up a `QuantumCircuit` object using Qiskit and set it for 2 qubits and 2 classical bits. 

In [2]:
#set up the circuits (do this later on)

## References

[1] Nielsen, M. A., & Chuang, I. L. (2011). Quantum Computation and Quantum Information: 10th Anniversary Edition. Cambridge University Press.

[2] Qiskit textbook, "name here" (exact page found [here](url here)). 